In [ ]:
!pip install -r requirements.txt

In [3]:
import pandas as pd
import sqlite3



In [4]:
def contains_kanji(s: str) -> bool:
    return any('\u4E00' <= ch <= '\u9FFF' for ch in s)

In [16]:
df_dict = {i: pd.read_csv(f'https://raw.githubusercontent.com/elzup/jlpt-word-list/refs/heads/master/src/n{i}.csv') for i in range(1,6)}
for n, df in df_dict.items():
    df.drop(df[~df.expression.apply(contains_kanji)].index, inplace=True)
    df['vocab_set'] = f'jlpt{n}'

df_all = pd.concat(df_dict.values(), ignore_index=True)
df_all.drop_duplicates(subset=['expression'], keep=False, inplace=True)
df_all.drop(columns=['tags'], inplace=True)
#Wrong reading in original
df_all.loc[(df_all.expression=='招待') & (df_all.reading=='しょうたいする'), 'reading']="しょうたい"
df_all.loc[(df_all.expression=='生活') & (df_all.reading=='せいかつする'), 'reading']="せいかつ"


In [17]:
with sqlite3.connect(r'..\..\android-app\app\src\main\assets\vocab.db') as conn:
    conn.execute("DROP TABLE IF EXISTS vocab;")

    conn.execute("""
    CREATE TABLE vocab (
        id INTEGER PRIMARY KEY NOT NULL,
        expression TEXT NOT NULL,
        reading TEXT NOT NULL,
        meaning TEXT NOT NULL,
        vocab_set TEXT NOT NULL
    );
    """)

    conn.commit()
    df_all.to_sql('vocab', conn, if_exists="append", index=False)
    conn.commit()


In [18]:
df_all[df_all.duplicated(['expression'])].sort_values(by=['expression'])

,expression,reading,meaning,vocab_set
